In [1]:
import pandas as pd # импортируем необходимые библиотеки
import numpy as np
import itertools
from itertools import combinations
import psycopg2.extras

In [2]:
def get_data(query):
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur =    conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data                      # создаем функцию для выгрузки результатов SQL-запросов

In [3]:
query='''
SELECT COUNT (DISTINCT s.user_id)
FROM final.carts s
JOIN final.cart_items c ON s.id = c.cart_id
WHERE state = 'successful' and resource_type = 'Course' '''

get_data(query)                      # считаем сколько клиентов покупали курсы

[{'count': 49006}]

In [4]:
query='''
SELECT COUNT (DISTINCT resource_id)
FROM final.cart_items
WHERE resource_type = 'Course' '''

get_data(query)                      # считаем сколько всего различных курсов

[{'count': 127}]

In [5]:
query='''
SELECT COUNT (DISTINCT c.id)
FROM final.carts s
JOIN final.cart_items c ON s.id = c.cart_id
WHERE state = 'successful' and resource_type = 'Course' '''

get_data(query)                      # считаем среднее количество купленных курсов на одного клиента

[{'count': 70581}]

In [6]:
query='''
WITH repeat_purchases AS
(
SELECT user_id,
COUNT(DISTINCT resource_id)
FROM final.carts c
JOIN final.cart_items i ON i.cart_id = c.id
WHERE state = 'successful' AND resource_type = 'Course'
GROUP BY 1
HAVING COUNT(DISTINCT resource_id) > 1
)

SELECT COUNT(DISTINCT user_id)
FROM repeat_purchases'''

get_data(query)                       # считаем сколько клиентов купили больше одного курса

[{'count': 12656}]

In [7]:
query='''
WITH users_with_repeat_purchases AS
(SELECT fc.user_id,
COUNT (ci.resource_id) count_course
FROM final.carts fc 
JOIN final.cart_items ci ON ci.cart_id = fc.id
WHERE fc.state = 'successful' AND ci.resource_type = 'Course'
GROUP BY fc.user_id
HAVING COUNT(DISTINCT ci.resource_id) > 1),
main_table AS
(SELECT *
FROM final.carts fc 
JOIN final.cart_items ci ON ci.cart_id = fc.id
WHERE ci.resource_type='Course' AND  fc.state='successful' )
SELECT DISTINCT uwmrp.user_id, mt.resource_id
FROM users_with_repeat_purchases uwmrp 
JOIN main_table mt ON uwmrp.user_id = mt.user_id
'''

df = pd.DataFrame(get_data(query))

df                                               # выгружаем список клиентов и курсов купленных ими более 1 раза

,user_id,resource_id
0,909757,356
1,583850,515
2,1559882,566
3,970967,679
4,1640443,566
...,...,...
34069,725913,553
34070,1092403,566
34071,180357,551
34072,201631,570


In [8]:
df.sort_values(by=['user_id', 'resource_id'], ascending=[True, True], inplace=True)

df.head()                      # сортируем датафрейм по возрастанию

,user_id,resource_id
7338,51,516
23178,51,1099
26621,6117,356
10771,6117,357
16620,6117,1125


In [9]:
# группируем курсы по пользовтелям
df_grouped = df.groupby('user_id')['resource_id'].apply(list).reset_index()

# создаем датафрейм для хранения пар
course_pairs_df = pd.DataFrame(columns=['user_id', 'resource_id'])

# генерируем все возможные комбинации курсов для каждого пользователя и заполняем ими датафрейм
for _, row in df_grouped.iterrows():
    pairs = list(itertools.combinations(row['resource_id'], 2))
    df_pairs = pd.DataFrame({'user_id': [row['user_id']] * len(pairs), 'resource_id': pairs})
    course_pairs_df = pd.concat([course_pairs_df, df_pairs], ignore_index=True)

course_pairs_df

,user_id,resource_id
0,51,"(516, 1099)"
1,6117,"(356, 357)"
2,6117,"(356, 1125)"
3,6117,"(357, 1125)"
4,10275,"(553, 1147)"
...,...,...
40012,2187601,"(765, 912)"
40013,2188926,"(515, 743)"
40014,2190141,"(756, 794)"
40015,2190141,"(756, 1185)"


In [10]:
course_pairs_df.nunique() # считаем количество различных пар курсов в покупках клиентов

user_id        12656
resource_id     3989
dtype: int64

In [11]:
course_pairs_count_df = pd.DataFrame(course_pairs_df['resource_id'].value_counts()).reset_index()

course_pairs_count_df.head() # записываем в датафрейм количество покупок для каждой пары курсов

,resource_id,count
0,"(551, 566)",797
1,"(515, 551)",417
2,"(489, 551)",311
3,"(523, 551)",304
4,"(566, 794)",290


In [12]:
# делим кортеж с парами на два отдельных столбца
course_pairs_count_df[['course_1', 'course_2']] = course_pairs_count_df['resource_id'].apply(pd.Series)

# удаляем столбец с кортежами
course_pairs_count_df = course_pairs_count_df.drop('resource_id', axis=1)

course_pairs_count_df.head()

,count,course_1,course_2
0,797,551,566
1,417,515,551
2,311,489,551
3,304,523,551
4,290,566,794


In [13]:
course_pairs_count_df['count'].describe() # смотрим распределение частоты покупок пар для определения нижней границы

count    3989.000000
mean       10.031838
std        26.355998
min         1.000000
25%         1.000000
50%         3.000000
75%         9.000000
max       797.000000
Name: count, dtype: float64

Как видно на распределении выше, 75 процентов пар были приобретены вместе менее 10 раз. Конечно для системы рекомендаций, хотелось бы использовать данные с большим числом количественных подтверждений совместной покупки курсов. С другой стороны, выбрасывать большую часть собранных данных тоже неправильно.
Поэтому я принял решение, выставить нижнюю границу на 5 процентом квантиле, который в нашем случае равен 1. Соответсвенно, если для какого-то из курсов будет лишь 1 совместная покупка, то данная рекомендация не будет считаться корректной и вместо этого будет рекомендован наиболее популярный курс. Впоследствии, с увеличением количества данных, нижняя граница может сдвинуться вверх, либо можно изменить квантиль ее расчета

In [14]:
lower_limit = course_pairs_count_df['count'].quantile(0.05) # устанавливаем нижнию границу на 5 процентном квантиле

lower_limit

1.0

In [15]:
query='''
SELECT resource_id, COUNT(cart_ID)
FROM final.carts fc 
JOIN final.cart_items ci ON ci.cart_id = fc.id
WHERE ci.resource_type='Course' AND  fc.state='successful'
GROUP BY resource_id
ORDER BY 2 DESC
'''

top_courses_df = pd.DataFrame(get_data(query))

top_courses_df    # выгружаем список наиболее популярных курсов для рекомендации при недостаточном числе совместных покупок

,resource_id,count
0,551,9570
1,566,6222
2,490,2712
3,794,2525
4,515,2049
...,...,...
121,902,5
122,1201,5
123,1182,5
124,1200,2


In [16]:
recommendations_df = top_courses_df.sort_values(by='resource_id').reset_index().drop(columns=['count', 'index'])

recommendations_df # подготавливаем датафрейм со списком курсов в который далее будут добавлены рекомендации

,resource_id
0,356
1,357
2,358
3,359
4,360
...,...
121,1188
122,1198
123,1199
124,1200


In [17]:
# Cоздаем пустые колонки для рекомендаций 
recommendations_df['option_1'] = None
recommendations_df['option_2'] = None

# итерируем по всем курсам в списке
for index, row in recommendations_df.iterrows():
    course_number = row['resource_id']
    
    # фиьтруем те пары где один из курсов соответствует текущему
    filtered_df = course_pairs_count_df[(course_pairs_count_df['course_1'] == course_number) | (course_pairs_count_df['course_2'] == course_number)]
    
    if filtered_df.empty or filtered_df.iloc[0]['count'] <= lower_limit:
        # проверяем количество совместных покупок на превышение нижней границы, в обратном случае используем наиболее популярный курс для рекомендации
        option_1 = top_courses_df.loc[0, 'resource_id']
        option_2 = top_courses_df.loc[1, 'resource_id']
    else:
        # сортируем отфильрованные пары по убыванию покупок
        sorted_df = filtered_df.sort_values('count', ascending=False)
        
        # находим 2 наиболее часто покупаемых совместно курса для рекомендации
        option_1 = sorted_df.iloc[0]['course_2'] if sorted_df.iloc[0]['course_1'] == course_number else sorted_df.iloc[0]['course_1']
        option_2 = sorted_df.iloc[1]['course_2'] if sorted_df.iloc[1]['course_1'] == course_number else sorted_df.iloc[1]['course_1']
    
    # добавляем полученные значения в датафрейм
    recommendations_df.at[index, 'option_1'] = option_1
    recommendations_df.at[index, 'option_2'] = option_2

recommendations_df

,resource_id,option_1,option_2
0,356,571,357
1,357,571,356
2,358,570,752
3,359,570,358
4,360,745,516
...,...,...,...
121,1188,1141,513
122,1198,571,912
123,1199,523,570
124,1200,551,566
